<a href="https://colab.research.google.com/github/bnbryan/hpml-project/blob/master/prune_on_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load the model

install library

In [ ]:
%pip install transformers
%pip install datasets

In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

Load pre-trained gpt2 model from library

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer.pad_token = tokenizer.eos_token

## Data pre-processing

In [ ]:
from datasets import load_dataset

# 加载 WikiText-103 数据集
dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="test")
# 分词
def tokenize_text(examples):
  return tokenizer(
        examples["text"],
        return_special_tokens_mask=True,
        truncation=True,
        max_length=512,
        padding='max_length'
    )

tokenized_dataset = dataset.map(tokenize_text, batched=True, remove_columns=['text'])
tokenized_dataset.set_format(type='torch', columns = ['input_ids', 'attention_mask'])

## Get the baseline perplexity

In [ ]:
def cal_perplexity():
    torch.cuda.empty_cache()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    model.eval()

    total_loss = 0
    total_tokens = 0

    # 创建 DataLoader
    dataloader = DataLoader(tokenized_dataset, batch_size=8)

    # 遍历数据集
    with tqdm(dataloader, desc="Evaluating", unit="batch") as pbar:
        for batch in pbar:
            torch.cuda.empty_cache()
            tokens = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)

            labels = tokens.clone()
            labels[attention_mask == 0] = -100

            with torch.no_grad():
                outputs = model(input_ids=tokens, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss

                num_active_tokens = attention_mask.sum().item()
                total_loss += loss.item() * num_active_tokens
                total_tokens += num_active_tokens
    
    perplexity = torch.exp(torch.tensor(total_loss / total_tokens))
    print(f"perplexity: {perplexity:.4f}")
    torch.cuda.empty_cache()
    return perplexity


In [ ]:
cal_perplexity()

# Pruning

In [ ]:
import torch.nn.utils.prune as prune

## Prune transformers

### functions to restore pruned layers

In [ ]:
def restore_pruned_layer(layer):
    if hasattr(layer, "weight_orig"):
        orig_weight = getattr(layer, "weight_orig")
        setattr(layer, "weight", orig_weight)
        delattr(layer, "weight_orig")
    if hasattr(layer, "weight_mask"):
        delattr(layer, "weight_mask")
        
def remove_hooks(layer):
    for hook_id in list(layer._forward_pre_hooks.keys()):
        del layer._forward_pre_hooks[hook_id]
        
def restore_block_n(block_num):
    for i in range(4 * block_num, 4 * block_num + 4):
        restore_pruned_layer(model.transformer.h[i].mlp.c_fc)
        restore_pruned_layer(model.transformer.h[i].mlp.c_proj)
        remove_hooks(model.transformer.h[i].mlp.c_fc)
        remove_hooks(model.transformer.h[i].mlp.c_proj)

### functions to prune layers

In [ ]:
def prune_mlp_layer(mlp, pruning_ratio=0.3):
    prune.l1_unstructured(mlp.c_fc, name="weight", amount=pruning_ratio)
    prune.l1_unstructured(mlp.c_proj, name="weight", amount=pruning_ratio)

def prune_block(block_num, pruning_ratio):
    for i in range(4 * block_num, 4 * block_num + 4):
        prune_mlp_layer(model.transformer.h[i].mlp, pruning_ratio)

### Prune, calculate perplexity, and restore pruned layers

In [ ]:
prune_block(0, 0.3)
cal_perplexity()
restore_block_n(0)